<a href="https://colab.research.google.com/github/hamiltonvalerio/DS_CNES_DATASUS/blob/main/CNES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import glob
import time
import requests
from io import StringIO

# SELECIONANDO NO GOOGLE DRIVE A TABELA DO MÊS 12/2020 DE NOMES DE EQUIPAMENTOS (tbEquipamento202012.csv) PARA FAZER UM JOIN COM O DATAFRAME de EQUIPAMENTOS TRAZENDO A NOMENCLATURA (Gama Camara, PET/CT)

In [ ]:
url_eq_auxiliar = 'https://drive.google.com/file/d/19Qikm47nw3cy7FL6QwZj_4XmVHzz4Xp6/view?usp=sharing'
path_eq_auxiliar = 'https://drive.google.com/uc?export=download&id='+url_eq_auxiliar.split('/')[-2]
df_eq = pd.read_csv(path_eq_auxiliar,sep=';')

filtro_eq = ['Gama Camara', 'PET/CT']

result_eq = df_eq[df_eq['DS_EQUIPAMENTO'].str.contains('|'.join(filtro_eq))]

result_eq

,CO_EQUIPAMENTO,CO_TIPO_EQUIPAMENTO,DS_EQUIPAMENTO
7,1,1,Gama Camara
66,18,1,PET/CT


# ARRAY DE ESTADOS PARA PESQUISA

In [ ]:
array_estados = np.array(["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"])
pd.options.display.max_columns = None

# MONTA O ARRAY DE LINKS DE EQUIPAMENTOS

In [ ]:
path_eq = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQ'
array_urls_eq = []
campos_eq = ['CNES','CPF_CNPJ','CODEQUIP']

for est in array_estados:
    array_urls_eq.append(path_eq+""+est+"_2020.csv")
    print(array_urls_eq)


['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAM_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAP_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/EQ/CSVs/EQA

# GERA O DATAFRAME DE EQUIPAMENTOS

In [ ]:
timeStartEQ = time.time()
dfs_EQ = [pd.read_csv(url,sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_eq) for url in array_urls_eq]
total_df_EQ = pd.concat(dfs_EQ, ignore_index=True)
tempoDeExecucaoEQ = (time.time() - timeStartEQ)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoEQ/60))

Tempo de execução em minutos: 1.591947372754415


In [ ]:
total_df_EQ['CODEQUIP'] = total_df_EQ['CODEQUIP'].astype(int)

In [ ]:
total_df_EQ.head()

,CNES,CPF_CNPJ,CODEQUIP
0,3006166,00000000000000,81
1,3006166,00000000000000,84
2,3006166,00000000000000,83
3,3006166,00000000000000,80
4,3382745,00000000000000,81


In [ ]:
total_df_EQ.shape

(8596372, 3)

In [ ]:
total_df_EQ.dtypes

CNES        object
CPF_CNPJ    object
CODEQUIP     int64
dtype: object

# FILTRA O DATAFRAME DE EQUIPAMENTOS COM OS FILTROS: coluna_codequip.values (1,18) - QUE CORRESPONDEM AOS CODIGOS DOS EQUIPAMENTOS: Gama Camara e PET/CT 

In [ ]:
coluna_codequip = result_eq.loc[:,'CO_EQUIPAMENTO'].astype(int)
coluna_codequip.values

array([ 1, 18])

In [ ]:
result_df_EQ = total_df_EQ.loc[total_df_EQ['CODEQUIP'].isin(coluna_codequip.values)]

In [ ]:
result_df_EQ.shape

(7343, 3)

In [ ]:
result_df_EQ

,CNES,CPF_CNPJ,CODEQUIP
921,2001977,00624104000177,1
1396,6225012,07616992000125,1
1538,6655653,11467937000198,1
2138,9467610,28685425000109,1
3609,2001977,00624104000177,1
...,...,...,...
8591389,2792451,25053117004828,1
8591436,2468972,25053117005123,1
8593585,7653948,06180514000152,1
8595290,5098246,07720890000155,1


In [ ]:
result_df_EQ.dtypes

CNES        object
CPF_CNPJ    object
CODEQUIP     int64
dtype: object

# MONTA O ARRAY DE LINKS ESTABELECIMENTOS

In [ ]:
path_st = 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/ST'
array_urls_st = []
campos_st = ['CNES','CPF_CNPJ','ALVARA']

for est in array_estados:
    array_urls_st.append(path_st+""+est+"_2020.csv")
    print(array_urls_st)

['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAM_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAL_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAM_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAP_2020.csv']
['https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STAC_2020.csv', 'https://datascience.ipen.br/~mario/CNESdados/200508_/Dados/ST/CSVs/STA

# GERA O DATAFRAME DE ESTABELECIMENTOS

In [ ]:
timeStartST = time.time()
dfs_ST = [pd.read_csv(urlst, sep=',', error_bad_lines=False, index_col=False, dtype='unicode', usecols = campos_st) for urlst in array_urls_st]
total_df_ST = pd.concat(dfs_ST, ignore_index=True)
tempoDeExecucaoST = (time.time() - timeStartST)
print('Tempo de execução em minutos: ' + str(tempoDeExecucaoST/60))

Tempo de execução em minutos: 2.2494387944539387


In [ ]:
total_df_ST.shape

(3581555, 3)

In [ ]:
total_df_ST

,CNES,CPF_CNPJ,ALVARA
0,3006166,00000000000000,NaN
1,3382745,00000000000000,NaN
2,3393984,00000000000000,NaN
3,3638685,00000000000000,NaN
4,5403669,00000000000000,NaN
...,...,...,...
3581550,7170149,00000000000000,36/2019
3581551,7170157,00000000000000,34/2019
3581552,7200447,00000000000000,35/2019
3581553,7970269,13038293000148,02/2018


# FAZ O MERGE (RIGHT JOIN NO SQL) USANDO COMO CHAVE O VALOR CNES E REMOVENDO OS DUPLICADOS

Aqui tenho um resultado de todas as empresas da base CNES que trabalham (ou tenham!?) os equipamentos GAMA CAMARA e PET/CT. Nesta lista podemos conferir se, pelo CNES, temos o Alvará.
Com base nesta lista é possível importar a base de Clientes da Cnen e fazer uma comparação utilizando o CNES ou CNPJ cadastrado. 

In [ ]:
pd.merge(total_df_ST, result_df_EQ, on="CNES", how="right").drop_duplicates()

,CNES,CPF_CNPJ_x,ALVARA,CPF_CNPJ_y,CODEQUIP
0,2001977,00624104000177,003 REGISTRO 001,00624104000177,1
121,6225012,07616992000125,00001239864,07616992000125,1
242,6655653,11467937000198,147,11467937000198,1
363,9467610,28685425000109,9038191,28685425000109,1
484,2005417,04710210000124,17071/2019,04710210000124,1
...,...,...,...,...,...
78506,2792451,25053117004828,028/2002,25053117004828,1
78627,2468972,25053117005123,019,25053117005123,1
78748,7653948,06180514000152,NaN,06180514000152,1
78869,5098246,07720890000155,007,07720890000155,1


In [9]:
url_lista_clientes_cnen = 'https://drive.google.com/file/d/1OvY1NsR0Bn8eDq2I28TRvwjQPEdPywAh/view?usp=sharing'
path_lista_clientes_cnen = 'https://drive.google.com/uc?export=download&id='+url_lista_clientes_cnen.split('/')[-2]
df_lista_clientes_cnen = pd.read_excel(path_lista_clientes_cnen, index_col=0)
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace(".","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("/","")
df_lista_clientes_cnen['CNPJ'] = df_lista_clientes_cnen['CNPJ'].str.replace("-","")
df_lista_clientes_cnen

,seq,CODIGORESP,CNPJ,RAZAOSOCIAL,UNIDADE,RESPONSAVEL,MATRICULACNEN,ENDERECO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
CODIGOCLIENTE,,,,,,,,,,,,,,
5002184,0,1,21554423000123,INSTITUTO ONCOLÓGICO LTDA (JUIZ DE FORA-MG),MEDICINA NUCLEAR,GUSTAVO SFFEIR C. DE MAGALHAES,16404,RUA SANTOS DUMONT,56,NaN,GRANBERY,JUIZ DE FORA,MG,36010510
5002672,0,1,33205964000125,CLINICA DE MEDICINA NUCL. VILLELA PEDRAS LTDA,MED NUCLEAR,IVAN DE SÁ VILELA PEDRAS,11595,RUA MEXICO,98,3 E 4 ANDAR,CENTRO,RIO DE JANEIRO,RJ,20031141
5009782,0,1,15166416000151,REAL SOCIEDADE PORTUGUESA DE BENEFICIENCIA 16 ...,MEDICINA NUCLEAR,EDMARIO ANTONIO G. COSTA,11672,"AV. PRINCESA ISABEL,",914,NaN,BARRA AVENIDA,SALVADOR,BA,40140901
5016363,0,1,75033308000132,CENTRO DE MEDICINA NUCLEAR DO PARANA LTDA S/C,MEDICINA NUCLEAR,TAIZA PARENTE ZUKOVSKI,15390,RUA PDE ILDEFONSO,158,NaN,BATEL,CURITIBA,PR,80240160
5017408,0,1,33804212000180,CLINICA SAO CARLOS S/A,SERV MED NUCLEAR,CARLOS AFFONSO FUSER,11663,RUA HUMAITA,296,NaN,BOTAFOGO,RIO DE JANEIRO,RJ,22261001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6251323,0,1,15126437001549,EMPRESA BRASILEIRA DE SERVIÇOS HOSPITALARES - ...,MEDICINA NUCLEAR,Viviane S. Parisotto Marino,14304,AVENIDA ALFREDO BALENA,110,NaN,SANTA EFIGENIA,BELO HORIZONTE,MG,30130100
6251374,0,1,09609235000230,HOSPITAL CENTRAL DO EXERCITO,MEDICINA NUCLEAR,AFONSO EDUARDO M MUZITANO,16239,RUA FRANCISCO MANUEL,126,NaN,BEFICA,RIO DE JANEIRO,RJ,20911270
6251412,0,1,15126437002006,EBSERH HUPAA-UFAL,MEDICINA NUCLEAR,ANDRE GUSTAVO SILVA PINO,17420,AV LOURIVAL MELO MOTA,S/N,NaN,TABULEIRO DOS MARTINS,MACEIO,AL,57072900
